<a href="https://colab.research.google.com/github/chakrateja70/claude-Agent/blob/main/ClaudeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://chatgpt.com/c/67a198f6-fd28-800c-9a08-0e0dd41e3ac4

# *Required Pacakges*

In [8]:
!pip install langchain pinecone-client anthropic voyageai langchain-voyageai python-dotenv pypdf langchain-community datasets


In [ ]:
# import os
# from dotenv import load_dotenv

# # Load API keys from environment variables
# load_dotenv()

# # Set API keys (replace with actual keys)
# os.environ["PINECONE_API_KEY"] = "your-pinecone-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-claude-api-key"
# os.environ["VOYAGE_API_KEY"] = "your-voyage-api-key"

In [3]:
from google.colab import userdata
userdata.get('PINECONE_API_KEY')
userdata.get('ANTHROPIC_API_KEY')
userdata.get('VOYAGE_API_KEY')

'pa-vesi5MxnW621-kuaB7Xksv43lXs5uLqR3PqHcS4jzFu'

In [22]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    pages = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        pages.extend(loader.load())
    return pages

folder_path = "/content/"
pages = load_documents(folder_path)
print(f"Loaded {len(pages)} documents from the folder.")


Unsupported file type: .config
Unsupported file type: .ipynb_checkpoints
Unsupported file type: sample_data
Loaded 28 documents from the folder.


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.split_documents(pages)
print(f"split the document into  {len(splits)} chunks.")

split the document into  31 chunks.


In [32]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

In [61]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_3deAHz_FHXzbg85r64jV3Fk6gN5H1Vd4y9195QDN2gg7tsbN9v77S2tXFiHwaQ8cP8gr4b")
index = pc.Index("agent")

index_name = "agent"

pc.create_index(
    name="agent",
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


# Define embedding model
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store embeddings in Pinecone
vectordb = PineconeStore.from_documents(
    documents=splits,  # Ensure `splits` contains the list of document chunks
    embedding=embedding_function,
    index_name="agent"

)

print(f"Data successfully stored in Pinecone under index '{index_name}'!")

Data successfully stored in Pinecone under index 'agent'!


In [110]:
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY") or getpass("Anthropic API key: ")

# chat completion llm
llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model_name="claude-3-sonnet-20240229",  # change "opus" -> "sonnet" for speed
    temperature=0.0
)

retriever = vectordb.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("RAG pipeline successfully set up!")

RAG pipeline successfully set up!


In [109]:
# User Query
query = "what is FY26?"

# Get Response from RAG System
response = rag_chain.invoke({"query": query})

# Check if response is empty
if not response.get("result"):
    print("No relevant information found in the knowledge base.")
else:
    print("Answer:", response["result"])


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}